# Training a classifier on a simple synthetic dataset

Here is a bunch of text - I hope it looks normal! (make stuff hidden)

This tutorial will demonstrate how to train a classifier with and without fairness regularization. 

In [1]:
# Required for notebook path to be at head of project for torch_fairness imports
import sys
sys.path.insert(0, os.path.abspath('../../..'))
import os
os.chdir('../../..')

In [2]:
import os
from tqdm import tqdm

import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

from torch_fairness.data import SensitiveMap
from torch_fairness.data import SensitiveTransformer
from torch_fairness.metrics import AccuracyEquality

In [3]:
def evaluate(model, features: torch.tensor, labels: torch.tensor, sensitive: torch.tensor, threshold = 0.5):
    fairness_measure = AccuracyEquality(sensitive_map=sensitive_map, threshold=threshold)
    metrics = {
        'accuracy': torch.mean(1.0*((1.0*(model(features)>threshold))==labels)).item(),
        'fairness': fairness_measure(pred=model(features), sensitive=sensitive, labels=labels).detach().numpy()
    }
    return metrics

## Data

The data used in this example is a simple synthetic dataset that was generated with a single binary sensitive attribute (e.g., minority, majority). Additionally, it was generated with class imbalance (majority >> minority) and measurement invariance (relationship between features and labels differs between groups). These two attributes ensure that a model that is trained without any form of resampling, loss reweighting, or regularization will produce predictions that are more accurate for the majority vs. minority group.

In [12]:
sensitive_cols = ['demo']
label_col = ['hired']
features_cols = ['X_0', 'X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6', 'X_7', 'X_8', 'X_9']
data = pd.read_csv(os.path.join('datasets', 'synthetic_binary.csv'))

The first step is to specify the sensitive attributes using a SensitiveMap. This object is used to keep to keep track of the sensitive groups, produce dummy coded versions of the sensitive variables, and match minority-majority groups in fairness metrics.

In [5]:
sensitive_map = SensitiveMap(
    {'name': 'demo', 'majority': 'majority', 'minority': ['minority']}, 
)
sensitive_transformer = SensitiveTransformer(sensitive_map=sensitive_map)
sensitive_transformer.fit(data[sensitive_cols])
dummy_sensitive = sensitive_transformer.transform(data[sensitive_cols])

In [6]:
train_x, test_x, train_s, test_s, train_y, test_y = train_test_split(
    torch.tensor(data[features_cols].values, dtype=torch.float32),
    dummy_sensitive, 
    torch.tensor(data[label_col].values, dtype=torch.float32), 
    shuffle=True, 
    train_size=0.7,
    random_state=1
)

## Train model without fairness objective

The first model will be our baseline and will not include any strategy to address the class imbalance. 

In [7]:
model = torch.nn.Sequential(
    torch.nn.Linear(len(features_cols), 1),
    torch.nn.Sigmoid()
)
criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
for epoch in tqdm(range(200)):
    optimizer.zero_grad()
    outputs = model(train_x)
    loss = criterion(outputs.squeeze(), train_y.squeeze())
    loss.backward()
    optimizer.step()

100%|██████████| 200/200 [00:00<00:00, 4818.91it/s]


In [8]:
train_metrics = evaluate(model, features=train_x, labels=train_y, sensitive=train_s)
test_metrics = evaluate(model, features=test_x, labels=test_y, sensitive=test_s)
print(f'Training: {train_metrics}')
print(f'Test: {test_metrics}')

Training: {'accuracy': 0.9371428489685059, 'fairness': array([0.13965851], dtype=float32)}
Test: {'accuracy': 0.8933333158493042, 'fairness': array([0.14627284], dtype=float32)}


## Train model with fairness objective

In this second model, we will include a fairness measure, AccuracyEquality, which uses the difference in accuracy between the minority-majority group as a regularization term.

In [9]:
fair_model = torch.nn.Sequential(
    torch.nn.Linear(len(features_cols), 1),
    torch.nn.Sigmoid()
)
fairness_measure = AccuracyEquality(sensitive_map=sensitive_map)
criterion = torch.nn.BCELoss()
fairness_measure = AccuracyEquality(sensitive_map=sensitive_map)
optimizer = torch.optim.Adam(fair_model.parameters(), lr=0.1)
for epoch in tqdm(range(100)):
    optimizer.zero_grad()
    outputs = fair_model(train_x)
    crit_loss = criterion(outputs.squeeze(), train_y.squeeze())
    fair_loss = fairness_measure(pred=outputs, sensitive=train_s, labels=train_y)
    loss = crit_loss + fair_loss.mean()
    loss.backward()
    optimizer.step()

100%|██████████| 100/100 [00:00<00:00, 1265.81it/s]


In [10]:
train_metrics = evaluate(fair_model, features=train_x, labels=train_y, sensitive=train_s)
test_metrics = evaluate(fair_model, features=test_x, labels=test_y, sensitive=test_s)
print(f'Training: {train_metrics}')
print(f'Test: {test_metrics}')

Training: {'accuracy': 0.8999999761581421, 'fairness': array([0.00272262], dtype=float32)}
Test: {'accuracy': 0.8500000238418579, 'fairness': array([0.05123568], dtype=float32)}


As expected, we observed a decrease in over-all accuracy but an increase in fairness for the accuracy disparity. The degree of trade-off can be controlled through modifying the weighting between the criterion and fairness loss.

## Conclusion

This tutorial hopefully provided a simple example of how a fairness regularized model can be trained and evaluated using this package.